In [ ]:
!nvidia-smi

Tue Jun 27 17:33:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
! pip install -q langchain transformers sentence_transformers llama-index


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 97.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.9/541.9 kB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.4 MB/s eta 0:00:00


In [ ]:
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex,GPTVectorStoreIndex, PromptHelper
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LLMPredictor, ServiceContext
import torch
from langchain.llms.base import LLM
from transformers import pipeline

In [ ]:
class customLLM(LLM):
    model_name = "google/flan-t5-large"
    pipeline = pipeline("text2text-generation", model=model_name, device=0, model_kwargs={"torch_dtype":torch.bfloat16})

    def _call(self, prompt, stop=None):
        return self.pipeline(prompt, max_length=9999)[0]["generated_text"]
    @property
    def _identifying_params(self):
        return {"name_of_model": self.model_name}
    @property
    def _llm_type(self):
        return "custom"


llm_predictor = LLMPredictor(llm=customLLM())


In [ ]:
hfemb = HuggingFaceEmbeddings()
embed_model = LangchainEmbedding(hfemb)


In [ ]:
# https://www.sunlife.ca/en/health/personal-health-insurance/basic-plan/
text1 = """
          Dear Customer,




We are happy to inform you that “INGO ARMENIA” Insurance CJSC continues to stay
the provider of health insurance of Simonyan Educational Foundation and its affiliated
companies employees.


The health insurance coverage, application procedure, the list of partner medical
facilities are presented in this brochure.


Contact us and we will take care of your health․


Please get acquainted with health insurance’s terms and conditions.



Please call INGO ARMENIA 24/7 Call Center in case of questions or for getting further
information:




                                  +374 (10) 59-21-21

WHAT IS COVERED

  1. AMBULANCE
  1.1. Full range services of ambulance
  1.2. Urgent medical transfers on the territory of the RA and / or NKR with specialized vehicles and
       professional medical personnel (exclusively by land vehicles);
  1.3. 24/7 medical consultations with professionals.


  2. OUT-PATIENT TREATMENT
  2.1. Medical consultations;
  2.2. Laboratory diagnostics;
  2.3. Instrumental diagnostics;
  2.4. Medical interventions in outpatient settings;
  2.5. Out-patient treatment for urgent therapeutic diseases or surgical diseases which does not require
        surgical treatment;
  2.6. Planned or urgent surgical interventions in out-patient conditions;
  2.7. Out-patient treatment for traumas (fractures, dislocations, soft tissue bruises, wounds or their
        combinations) in out-patient conditions;
  2.8. Out-patient treatment for burns, frostbites and electrical injuries;
  2.9. Out-patient treatment for infectious diseases;
  2.10. Out-patient treatment of acute poisonings;
  2.11. Compensation for the cost of medications prescribed in outpatient settings;
  2.12. Medical supplies: syringes, intravenous drip systems, bandages, cotton (assigned in outpatient
        conditions);
  2.13. Medical supplies: immobilization appliances, plastic mesh of hernia up to AMD 50,000 (assigned
        in inpatient and/or outpatient conditions);
  2.14. Vaccinations for travelling abroad;
  2.15. One-time consultation of a psychologist for each Insured person up to AMD 7,000;
  2.16. Home visit services in Yerevan (from the outpatient medical center mentioned by the Insurer);
  2.17. Organizing home-based examinations and other laboratory instrumental investigations when an
        insured person is unable to seek medical care due to health problems.
  2.18. Reimbursement of maximum of AMD 50,000 per case for outpatient only medical expenses
        incurred at doctor’s prescription, which are based on positive Covid-19 PCR test result, or
        pneumonia specifi to Covid-19, diagnozed according to CT scan. Medical expenses are limited to
        medication, examinations, PCR test.

  3. IN-PATIENT TREATMENT
  3.1. In-patient treatment for urgent therapeutic or surgical diseases which does not require surgical
       treatment;
  3․2.   Surgical treatment;
  3․3.   During in-patient treatment laboratory and instrumental diagnostics implemented by doctor’s
         prescription and written justification;
  3․4.   In-patient treatment for infectious diseases;
  3․5.   In-patient treatment for traumas (fractures, dislocations, soft tissue bruises, wounds or their
         combination)
  3․6.   In-patient treatment for burns, frostbites and electrical injuries;
  3․7.   In-patient treatment for acute poisonings;




                                                                                                       2

3․8.    Intensive therapy and reanimation requiring urgent treatment conditions (not included in state
        order programs 1);
3․10.   Metal constructions used in trauma surgery (except endoprostheses);
3․10.   Compensation for the cost of prescribed medications for in-patient treatment;
3.11.   Medical supplies: syringes, intravenous drip systems, bandages, cotton (in-patient).
        In cases mentioned in 3.10 and 3.11, the cost of those medications and/or medical supplies that
        are substantiated by a certificate issued by the medical institution, but not provided by the medical
        institution, nor included in the cost of treatment, should be reimbursed (sealed by the medical
        institution);
3.12.   Surgical treatment of nasal septum deformations, nasal passages, nodal cavities up to
        AMD 150,000 (the price includes expenses for all pre-surgical laboratory and
        instrumental examinations); Surgical treatment of nasal septum deformations, nasal
        passages, nodal cavities up to AMD 150,000 (the price includes expenses for all pre-
        surgical laboratory and instrumental examinations).
        The limit of indemnity for the same insurance accident mentioned in the clause 3.12 of
        the Contract are applicable only within the scope of the Contract in force as of the date
        of the accident and the provisions of the new Contract cannot be applied to previous
        insurance accidents in which the provisions of the invalid contract have already been
        applied.
3.13.   Compensation for hospital accommodation up to AMD 20,000 for each day.


4. VERTEBROLOGY
4.1.   Drug treatment and diagnosis of exacerbations of degenerative dystrophic disorders with
       neurological symptoms;
4.2.   Physiotherapy methods following drug treatment of dystrophic spinal degenerative disorders:
       electrophoresis, phonophoresis, UHF therapy, amplipulse therapy, ultraviolet irradiation therapy,
       therapeutic massage: for all the services listed herein, 2 course, each course 10 sessions within
       the contract period;
4.3.   Diagnostics of degenerative dystrophic deteriorations of spine, including MRI, if any of the
       following indications exist:
     • neurological deficiency (paresis of nerve, neurological signs of spinal cord deterioration), which
       is described by a neurologist or a neurosurgeon and is reflected with clear neurological deficit.
       The described case can also be confirmed by a neuromyographic examination.
     • back pain and inflammatory symptoms, in the case of absence of internal organs deteriorations
     • non-mechanical pain (is not changed during a day out of positions or movements, particularly in
       case of night pains)
     • mechanical/non-mechanical pain, which continues for 4-6 weeks (particularly after medicinal
       and rehabilitation treatments)
     • postsurgical (surgery of spine, spinal cord) persistent pain or neurological deficit.
       In all cases, Insurer can organize a double neurologist’s consultation on the neurological status
       assigned by a doctor who prescribed MRI examination at the expense of the Insurer and the
       specialist mentioned by the latter.
4.4.   Surgical treatment of spinal hernia.



1
 For this contract, the term “state order” means services that are included in the Government of the Republic of
Armenia 04․03․2004, decision No. 318-N “On Free and Privileged Medical Assistance and Service Guaranteed by
the State” and amendments/additions thereto.

                                                                                                               3

5. PHYSIOTHERAPY
5.1. Electrotherapy: Electrophoresis, phonophoresis, UHF therapy, amplipulse therapy, ultraviolet
     irradiation therapy, therapeutic massage. For all the services listed herein, 2 course, each course
     10 sessions within the contract period;
5.2. Post-traumatic physiotherapeutic and rehabilitation treatment up to AMD 100,000;
5.3. The physiotherapy mentioned above in 5․1 and 5․2 is not applicable to cases of spine that are
     regulated by paragraph 4 of this Program.


6. OPHTHALMOLOGY
6.1. Surgical treatment of ophthalmic diseases;
6.2. Non-surgical treatment of therapeutic or surgical eye diseases;
6.3. Reimbursement of acquisition costs of optical lenses, spectacle frames and optical glasses in case
     of vision changes (at least 0.5) up to AMD 25,000
6.4. Surgery, whose prime purpose is to correct defective eyesight when myopia/hyperopia is OU (+)
     օr (-) 2,5d or more, for which Insurer covers at 50%.


7. DENTISTRY
7.1. Dental coverage operates in accordance with the Dental service limits specified in 7.2;
7.2. Limits of indemnity on dental services

 Service                                                                Limits of Indemnity

 X-ray (if cases are reimbursable)

 Tooth’s crown reconstruction

                                                      simple
                                                    average
                                                  complicated
 Endodontics

                                     single-rooted tooth
                                   two-rooted root tooth
                                      three-rooted tooth
                                  teeth roots retreatment
 Treatment and prevention of periodontitis (once a year)            AMD 80,000 for all the listed
                                                                   services for one person during
     removal of dental plaque, polishing (including Air-Flow)           the contract period
                                               Gum treatment
 Surgery

                                      tooth simple extraction
                                    tooth complex extraction
                                     wisdom tooth extraction
                                    retained tooth extraction
                                                 apexectomy
                                            abscess drainage
 Dental orthopedics (the limit also includes the cost of
 therapeutic or surgical treatment of the same teeth preceding
 orthopedic prosthesis)


                                                                                                       4

  Implantology (The limit also includes pre-implantation
 preparations, including the cost of the bone and its placement
 surgery)



8. PREGNANCY AND CHILDBIRTH
8.1․   Coverage for normal and pathological pregnancy expenses which are out of scope of state order
       (in-patient and/or out-patient cases) with the limit of AMD 100,000;
8.2․   Coverage for child delivery (including child delivery through cesarean section) which are out of
       scope of state;
8.3.   For the coverage mentioned in 8.2 point, 9 months waiting period is applied for new insured
       people;
8.4.   In the case of pregnancy and childbirth, compensation of the value for the hospital room shall be
       within the limits set out in points 8.1 and 8.2; the value of the hospital room specified in 3.13 is
       not applicable in this case.
8.5.   The limits of indemnity for the same insurance accident mentioned in clauses 8.1 and 8.2 of the
       Contract are applicable only within the scope of the Contract in force as of the date of the accident
       and the provisions of the new Contract cannot be applied to previous insurance accidents in which
       the provisions of the invalid contract have already been applied.


9. ONCOLOGY
9.1.    Surgical treatment of cancerous diseases;
9.2. Non-surgical complex one-time in-patient treatment of cancerous diseases - chemotherapy, radiation
     therapy.

10. CARDIOLOGY
10.1. Cardio-surgery (open hurt surgery);
10.2. Cardio-surgery (percutaneous coronary intervention);
10.3. Angioplasty with drug-eluting balloon;
10.4. Aortic valve replacement (only urgent surgeries of aortic aneurysm exfoliations are covered);
10.5. The cases listed in point 10 and its sub-points are compensated, only if they are not included in
      the “Life Stent Program” of the RA’s MoH.


11. NEUROSURGERY
11.1. Neurosurgical surgeries (open hurt surgery);
11.2. Neurosurgical surgeries (percutaneous coronary intervention) up to AMD 1,500,000.


12. CHRONIC DISEASES
12.1. Treatment of acute conditions of chronic diseases
12.2. All non-surgical services mentioned in points 2, 3, 6 and 10 or their sub-points under this
       Agreement shall be compensated at the acute or aggravating stage of the disease;
12.3. Surgical treatment of chronic diseases, which leads to full recovery of disease or condition (planned
       surgery);
12.4. Compensation for the cost of dynamic control of chronic diseases not mentioned in the "General
       Exceptions" clause of this Program (consulting and examinations), including oncological diseases
       up to AMD 100,000.


                                                                                                          5

13. ANNUAL HEALTH CHECK-UP
13.1. Routine annual health check up is provided once in a Policy period with the following scope:
Specialist consultations:
     General practitioner (pediatrist)
     Ophthalmologist (including visus test)
     Dentist
     Gynecologist/Urologyst
Laboratory examination:
     General blood test with leykoformula
     Urine general test
     Glucose analysis
     Lipide profile (total cholesterol, high density lipoproteins, low density lipoproteins, triglycerides)
     PAP test (for women)
     PSA test (for men above 45y.)
Instrumental examination:
     Electrodiagram (EKG)
     Ultrasound of abdominal organs and small pelvis
     Ultrasound of breast, thyroid gland.

13.2.    Routine annual health check ups can be conducted at any of the following medical facilities:
    •   “Vardanants” MC
    •   “Shengavit” MC
    •   “Medline” MC
    •   “Elite-Med” MC
    •   "Heratsi" Hospital Complex N 1

14. TRADITIONAL MEDICINE
  In cases when scientific medicine may not be applied for any reason, this Policy covers reasonable and
  customary charges for the following medical services:

  - acupuncture;
  - aromatherapy.


15. PERSONAL ACCIDENT INSURANCE
15.1. This contract also provides for a one-time compensation in the amount of AMD 1,000,000 in
      case of death of the Insured person as a result of an accident.


16. GENERAL EXCEPTIONS
16.1. Unless otherwise provided by the Contract or the Rules (as well as the Appendices or Programs
        to the Rules), the following illnesses, incidents, types of medical care, research, conditions and
        other services, and any costs thereof (including any costs are not subject to compensation
        laboratory and / or instrumental research, medical advice and medication), irrespective of
        whether they are separately covered by insurance coverage;
16.1.1. all pre-existing illnesses, injuries, and medical conditions prior to the entry into force of the
        Insurance Contract, whether or not they are declared, irrespective of their knowledge of the
        existence of a particular illness with the Insured person, when the disease has been objectively
        arising;



                                                                                                               6

16.1.2. Sexually transmitted diseases (syphilis, gonorrhoea, chlamydiosis, bacterial vaginosis
         (gardnereliosis), trichomoniasis, mycoplasmosis, ureoplasmosis, genital fungal lesions
         (candidiasis), cytomegalovirus, anal, genital herpes, herpes zoster etc.);
16.1.3. Any method of surgical correction of visual acuity, cataract, keratoconus;
16.1.4. diseases, conditions requiring dynamic control, supportive treatment, except for cases covered
         by this Plan;
16.1.5. osteopenia, osteoporosis, hypercholesterolemia
16.1.6. Methods of non-surgical treatment of oncological diseases and their associated complications not
         specified in the Plan;
16.1.7. Congenital anomaly, developmental defects (excluding surgical treatment of nasal septum
         deformations, nasal passages, nodal cavities), genetic, hereditary, chromosomal diseases;
16.1.8. Spinal, intervertebral disks degenerative deteriorations, hernias, osteochondrosis, scoliosis,
         kiphosis, etc., except for cases covered in the Plan;
16.1.9. Diseases or conditions requiring extracorporeal blood purification, including renal failure;
16.1.10.         infertility (primary, secondary), sexual dysfunction, contraceptive methods, artificial
         insemination, abortion (except for abortion with medical instructions), sex hormone dysfunction
         (dysmenorrhea, other hormonal abnormalities), reproductive abnormalities;
16.1.11. Lung emphysema, pneumosclerosis, multiple bronchiectasis
16.1.12.HIV/AIDS, hepatitis B, C, D, E, F, its consequences, complications, liver cirrhosis;
16.1.13.         disorders that have no objective symptoms of nervous system, mental disorders,
         borderline conditions - neuroses, sticky states, etc.
16.1.14.Detection of epilepsy, infantile cerebral paralysis, sleep and/or movement disorders, or any of its
         treatment modalities;
16.1.15.         Physiotherapeutic procedures not included in the Plan, non-scientific (alternative)
         diagnostic, treatment methods: homeopathy, phytotherapy, aromatherapy, hydrotherapy, mud
         cure, bio resonance diagnostics, diagnostics with Foll method, iridology, magneto therapy,
         energymedicine, scenar therapy, manual therapy, traditional systems of health improvement,
         experimental, exploratory treatment, telemedicine related services, extra corporal treatment;
16.1.16.         Cases, situations in which the documents required by the Rules for obtaining
         compensation were not present and/or were incomplete;
16.1.17.Cosmetic surgery, plastic, reconstructive surgery. Except where such treatment, surgery is
         required as a result of an injury during the Policy period;
16.1.18.         neoplasms of derma, mucous tunic, birthmarks, polypus, papilloma, buttercups, etc.
16.1.19.         Medical, paramedical equipment, obtaining, placement, replacement of cardiac rhythm
         stimulating, regulating equipment, cardio rhythm disorder revealing, regulating operations
         (electrophysiological examinations, electroablation, etc.), cardioverter-defibrillators. Hearing
         aids. Implants, as well as other corrective medical devices, appliances, including adjustment,
         obtaining and installation costs. Prothetics, endoprothetics, all types of prosthetics, preparation
         to them, except for the cases, when this treatment is necessary as a result of injury occurring
         during the Policy period. Exceptions of this point are cases covered by the Plan.
16.1.20.         Vaccinations, except for cases covered in the Plan;
16.1.21.Any expenses incurred due to failure, refusal to follow doctor’s prescription
16.1.22.         Weight correction, speech defects correction
16.1.23.         Preventive, sanitary arrangements, immunoprophylaxis, training apparatus, solarium,
         water treatments, etc.
16.1.24.         Therapeutic food, nutrition, biological additives (vitamins, etc.), care accessories, mineral
         waters, drugs not licensed by RA’s MoH Drug Inspection, care products, therapeutic cosmetics;
16.1.25.         Tuberculosis, poliomyelitis
16.1.26.         Allergodiagnostics, scratch test


                                                                                                             7

16.1.27.        Treatment, situations covered by State sponsored programs (the exception applies only
        inpatient cases)
16.1.28.        Dermatomycoses, mycoses of nail, soft tissue, psoriasis, eczema
16.1.29.        all cases in which the Insured person after receiving the medical services fails to re-
        examine at the Insurer’s request within 10 calendar days;
16.1.30.        Systemic autoimmune diseases, systemic diseases of connective tissue (systemic Lupus
        erythematosus, rheumatic arthritis, arthroses, dermatomyositis, systemic vasculitis (modulated
        periarteritis, hemorrhagic vasculitis), amyloidosis;
16.1.31.        During the first contract period coronary artery bypass surgery, coronary stenting of
        vessels with diameter decrease by 45% and more․ Neurosurgery via laser intervention;
16.1.32.        Within the dental coverage, treatment of teeth with caries, teeth under orthopedic crowns
        or bridges, teeth which need refilling or have defects of filling according to pre-existing check-
        up results, as well as aesthetic treatment, experimental methods and cosmetic dentistry, OPG,
        gum treatment, dental orthodontics, teeth orthopedics, teeth implantation;
16.1.33.        Medical, other services, not mentioned in the Contract
16.1.34.        The treatment (including medicinal treatment), means of treatment, diagnosis, which are
        not recognized as necessary for a certain disease, injury or health condition by registries of
        international medical associations or RA. Insurer has the right to make inquiries to specialists
        having appropriate qualifications (chief specialists of Ministry of Health, is necessary) for the
        necessity of provided service, medicinal treatment, done, prescribed examinations, take their
        conclusions as basis;
16.1.35.        Costs of using alcohol, drugs, toxic substances and other psychotropic substances,
        including car accidents.

16.2.   All exceptions listed in point 15 are general and applicable to all Programs, unless otherwise
        provided in the Program or Contract.
16.3.   In addition to the foregoing exemptions, additional exemptions or refusals of reimbursement
        may be specified for each Program as set forth in the relevant Appendices to the Rules or the
        Contract.




                                                                                                         8

HOW TO GET
COMPENSATION
Option 1: Direct payment to partner medical centers: We cooperate with more than 100
partner medical centers in RA and NKR, the list of which can be found here:
https://ingoarmenia.am/uploads/2022/05/Med-centers-arm.pdf
If you make a claim with this procedure you are free of bringing documentation and making
payments. INGO ARMENIA will do it.

Option 2: Paying compensation to Insured person: When Insured person visits a doctor
without informing INGO ARMENIA about his/her intention, in this case only, he/she provides the
documents to INGO ARMENIA in either way:

                       by following this link

                        personalcontact@ingoarmenia.am

     Medical document: reference form or epicrisis
     Financial document: receipt with breakdown, payment receipt, invoice, etc.

In order to receive insurance compensation, the insured person is obliged to submit to the
insurer the application for insurance compensation and the necessary documents for receiving
insurance compensation not later than 30 (thirty) days after the accident. In some cases, this
period may be extended for up to 3 (three) months. Documents for drugs should be provided
no more than 1 month following the date of prescription.



      You can apply for insurance services even if you have not received health insurance
      wallet card.




           YOU CAN DISCUSS YOUR TREATMENT PROCEDURE AND THE QUALITY OF IT WITH YOUR
               DOCTOR-EXPERT TAGUHI MATEVOSYAN WITH (010) 59 21 21 PHONE NUMBER
                                                                                             9

REASONABILITY

Reasonable and customay charges applied to the medical expenses stipulated under this contract
provided that the treatment is recognized by the local medical authorities as essential to the treatment
of the disease, injury or medical condition.



Within the scope of the Contract Reasonable price for treatment of any desease is considered to be
and is subject to compensation:

       I. the price of the method of treatment defined in agreement signed between Insurer and the
          Basic medical center, in case of receiving treatment in this certain Basic medical center,
      II. the price of the method of treatment defined in agreement signed between Insurer and any of
          Basic medical center (with Insured person’s choice), in case of receiving method of treatment
          with the initiative of Insured person in other medical center not included in the list of Basic
          medical centers,
     III. the price defined by medical center maximum specialized in the method of desease treatment,
          taking into account the provision of discounts for Insurer (if available) as well, if the method of
          desease treatment is not possible in the certain Basic medical centers.

In any case, insurance compensation paid within the scope of the Contract cannot exceed actual expenses
(paid amounts).



Within the scope of the Contract while determining the Reasonable price of any method of treatment
Insurer has the right to consider the methods of treatment the same (identic), if they are carried out
for the same diagnose and have the same result. The sub-types, modifications, qualifications of treating
specialists, academic degree, working experience, as well as the use of additional equipments,
substances, lights or rays while treating or treatment with other non-significant differences are not
considered as other method of treatment.



In each case, if the treatment expenses of Insured person, prices of provided medical aid and services
(including medicine, medical supplies, etc.) exceed the Reasonable price, Insured person should pay the
exceeding sum to Reasonable price on his/her expenses to medical center for receiving the treatment,
services or medicine/medical supplies.



The above mentioned points are exceptionally applied for in-patient treatment cases.




                                                                                                           10

The list of Basic medical centers

      “Erebuni” Medical center
      “Heratsi” hospital complex №1 (Yerevan State Medical University)
      "Muratsan" hospital complex clinic (Yerevan State Medical University)
      Mary Nupar ophthalmological clinic (Yerevan State Medical University)
      Center of modern surgery
      Haematology center after professor R.Yeolyan CJSC
      Nork Republican infectious clinical hospital
      Ophthalmological Center after S.V.Malayan Eye Clinic CJSC
      Kanaker-Zeytun MC
      National Oncological Center named after V.A.Fanarjyan CJSC
      "Elite-Med" MC
      Research center of maternal and child health protection
      Avangard Med Co. Ltd (in connection with surgical treatment of internasal septum)
      Nork Marash medical center
      Arabkir Joint Medical Center & Institute of Child and Adolescent Health
      Research center of radiation medicine and burns
      “Armenia” Republican medical center
      “Shengavit” medical center
      “Beglaryan” MC
      “Medline” MC
      "NEW-MED" Urology Center
      "Surb Astvatsamayr" MC




                                                                                           11

Thank you for trusting us your health



                                        12





Questions and answers

Q: What services are covered under the health insurance plan?
A: The health insurance plan covers a range of services, including ambulance services, outpatient treatment, in-patient treatment, vertebrology, physiotherapy, ophthalmology, dentistry, pregnancy and childbirth, oncology, cardiology, neurosurgery, treatment of chronic diseases, annual health check-ups, traditional medicine, and personal accident insurance.

Q: Are ambulance services covered under the health insurance plan?
A: Yes, the health insurance plan provides full range services of ambulance, urgent medical transfers within the territory of the RA and/or NKR with specialized vehicles and medical personnel, and 24/7 medical consultations with professionals.

Q: What outpatient treatments are covered under the health insurance plan?
A: The health insurance plan covers medical consultations, laboratory diagnostics, instrumental diagnostics, medical interventions in outpatient settings, surgical interventions in outpatient conditions, treatment for traumas, burns, and infectious diseases, vaccinations for traveling abroad, psychologist consultations, home visit services in Yerevan, and reimbursement of medical expenses related to COVID-19.

Q: What in-patient treatments are covered under the health insurance plan?
A: The health insurance plan covers in-patient treatment for urgent therapeutic or surgical diseases, surgical treatments, laboratory and instrumental diagnostics, treatment for infectious diseases, traumas, burns, acute poisonings, intensive therapy and reanimation, metal constructions used in trauma surgery, compensation for prescribed medications, and reimbursement of hospital accommodation costs.

Q: Is dental coverage included in the health insurance plan?
A: Yes, the health insurance plan includes dental coverage. It covers various dental services such as tooth crown reconstruction, endodontics, periodontitis treatment, dental surgery, dental orthopedics, and implantology within specified limits of indemnity.

Q: Does the health insurance plan cover pregnancy and childbirth expenses?
A: Yes, the health insurance plan provides coverage for normal and pathological pregnancy expenses that are out of the scope of the state order. It also covers childbirth, including cesarean section, with certain waiting period requirements for new insured individuals.

Q: Are cancer treatments covered under the health insurance plan?
A: Yes, the health insurance plan covers surgical treatment of cancerous diseases and non-surgical complex in-patient treatment such as chemotherapy and radiation therapy.

Q: What are the coverage options for cardiology-related treatments?
A: The health insurance plan covers various cardiology treatments, including cardio-surgery, angioplasty, aortic valve replacement, and certain urgent surgeries related to aortic aneurysm exfoliations. However, coverage for these treatments is provided only if they are not included in the "Life Stent Program" of the Republic of Armenia's Ministry of Health.

Q: Is there coverage for neurosurgical procedures?
A: Yes, the health insurance plan covers neurosurgical surgeries, both open-heart surgeries and percutaneous coronary interventions, up to a specified limit.

Q: Are there any general exceptions or limitations to the coverage provided by the health insurance plan?
A: Yes, there are general exceptions and limitations mentioned in the contract. These exceptions include specific illnesses, incidents, types of medical care, research, conditions, and services that are not covered by the insurance plan, regardless of whether they are separately covered. The details of these exceptions are provided in the contract and its associated rules.



Q: What services are covered under ambulance coverage?
A: The ambulance coverage includes a full range of ambulance services, urgent medical transfers within the Republic of Armenia and Nagorno-Karabakh (by land vehicles only), and round-the-clock medical consultations with professionals.

Q: What does out-patient treatment cover?
A: Out-patient treatment includes medical consultations, laboratory and instrumental diagnostics, medical interventions in outpatient settings, treatment for various conditions such as therapeutic diseases, surgical diseases (non-surgical treatment), traumas, burns, infectious diseases, acute poisonings, and more. It also covers the cost of prescribed medications, medical supplies, vaccinations for travel, psychological consultations, and home visit services in Yerevan.

Q: What is covered under in-patient treatment?
A: In-patient treatment covers urgent therapeutic or surgical diseases (non-surgical treatment), surgical treatment, laboratory and instrumental diagnostics, treatment for infectious diseases, traumas, burns, acute poisonings, and intensive therapy. It also includes the cost of prescribed medications, medical supplies, surgical treatments for nasal deformations, and compensation for hospital accommodation.

Q: Does the insurance plan cover vertebrology services?
A: Yes, the insurance plan covers drug treatment, diagnosis, and physiotherapy methods for degenerative dystrophic disorders with neurological symptoms. It also covers the diagnostics of spinal degenerative disorders, including MRI scans, and surgical treatment of spinal hernia.

Q: What physiotherapy services are included?
A: The physiotherapy coverage includes services such as electrophoresis, phonophoresis, UHF therapy, amplipulse therapy, ultraviolet irradiation therapy, and therapeutic massage. Additionally, it covers post-traumatic physiotherapeutic and rehabilitation treatment.

Q: What dental services are covered by the insurance plan?
A: The dental coverage includes various services such as X-rays, tooth crown reconstruction, endodontics, treatment and prevention of periodontitis, surgeries (tooth extraction, wisdom tooth extraction, etc.), dental orthopedics, implantology, and more. The coverage limits are specified for each service.

Q: Does the insurance plan cover pregnancy and childbirth expenses?
A: Yes, the insurance plan covers both normal and pathological pregnancy expenses that are not covered by the state order. It also covers the expenses related to child delivery, including cesarean section, with a waiting period of 9 months for new insured individuals. The coverage includes the compensation of hospital room expenses within the specified limits.

Q: Are cancer treatments covered under the insurance plan?
A: Yes, the insurance plan covers surgical treatment of cancerous diseases as well as non-surgical complex in-patient treatments such as chemotherapy and radiation therapy for cancer.

Q: What cardiac treatments are covered by the insurance plan?
A: The insurance plan covers a range of cardiac treatments, including cardio-surgery (open heart surgery and percutaneous coronary intervention), angioplasty with drug-eluting balloon, and aortic valve replacement. However, urgent surgeries for aortic aneurysm exfoliations are covered. It also excludes procedures covered by the "Life Stent Program" of the Ministry of Health.

Q: Are neurosurgical surgeries covered by the insurance plan?
A: Yes, the insurance plan covers neurosurgical surgeries, both open heart surgery and percutaneous coronary intervention, up to a specified limit of AMD 1,500,000.

Q: Does the insurance plan cover mental health services?
A: Yes, the insurance plan includes coverage for mental health services. It covers consultations with psychiatrists, psychologists, and psychotherapists, as well as psychotherapy sessions and prescribed psychiatric medications.

Q: What diagnostic tests and screenings are covered under preventive care?
A: Preventive care coverage includes a wide range of diagnostic tests and screenings such as blood tests, cholesterol checks, mammograms, pap smears, colonoscopies, bone density scans, and vaccinations. These services aim to detect potential health issues early on and promote overall wellness.

Q: Are alternative treatments like acupuncture and chiropractic care covered by the insurance plan?
A: Yes, the insurance plan offers coverage for alternative treatments, including acupuncture and chiropractic care. It includes a specific number of sessions per year for each type of treatment.

Q: Are vision and eye care services covered by the insurance plan?
A: Yes, the insurance plan provides coverage for vision and eye care services. This includes annual eye exams, prescription eyeglasses or contact lenses, and a portion of the cost for vision correction procedures such as LASIK or PRK surgery.

Q: Does the insurance plan cover pre-existing conditions?
A: Pre-existing conditions are generally covered by the insurance plan, subject to certain conditions and waiting periods. The specifics may vary, so it's recommended to refer to the policy document or contact the insurance provider for more information.

Q: Are emergency medical evacuations covered by the insurance plan?
A: Yes, the insurance plan includes coverage for emergency medical evacuations. If you require urgent medical treatment that is not available locally, the insurance will cover the cost of transportation to the nearest appropriate medical facility.

Q: Does the insurance plan offer coverage for prescription medications?
A: Yes, the insurance plan covers a wide range of prescription medications. The coverage may vary based on the specific drug and its classification. It's advisable to consult the formulary or contact the insurance provider for a detailed list of covered medications.

Q: Are preventive vaccinations for travel included in the insurance plan?
A: Yes, the insurance plan covers preventive vaccinations for travel. It includes the cost of recommended vaccines to protect against various diseases prevalent in specific travel destinations.

Q: Is there coverage for home healthcare services?
A: Yes, the insurance plan provides coverage for home healthcare services. This includes skilled nursing care, physical therapy, occupational therapy, and medical equipment or supplies needed for home-based treatment.

Q: Does the insurance plan offer coverage for organ transplants?
A: Yes, the insurance plan covers the expenses related to organ transplants. This includes the cost of the transplant surgery, post-operative care, and necessary medications.


What services are covered under ambulance coverage?

Full range services of ambulance, urgent medical transfers, and 24/7 medical consultations.
What does out-patient treatment cover?

It covers medical consultations, laboratory and instrumental diagnostics, medical interventions, surgical interventions, treatment for traumas, infectious diseases, burn treatment, and more.
What is the reimbursement limit for outpatient medical expenses related to Covid-19?

The reimbursement limit is a maximum of AMD 50,000 per case.
What treatments are covered under in-patient treatment?

It covers urgent therapeutic or surgical diseases, laboratory and instrumental diagnostics, treatment for infectious diseases, traumas, burns, and more.
What is the coverage for vertebrology?

It covers drug treatment, diagnosis of degenerative dystrophic disorders, physiotherapy methods, and surgical treatment of spinal hernia.
What services are included in physiotherapy coverage?

It includes electrotherapy, post-traumatic physiotherapeutic and rehabilitation treatment.
What treatments are covered under ophthalmology?

It covers surgical and non-surgical treatment of eye diseases, as well as reimbursement for optical lenses and glasses.
How does dental coverage work?

Dental coverage includes various services such as X-ray, crown reconstruction, endodontics, periodontitis treatment, surgery, orthopedics, implantology, and more.
What does the coverage for pregnancy and childbirth include?

It covers normal and pathological pregnancy expenses, child delivery (including cesarean section), and hospital room expenses.
What treatments are covered under oncology?

It covers surgical treatment of cancerous diseases and non-surgical complex treatment such as chemotherapy and radiation therapy.
What treatments are covered under cardiology?

It covers cardio-surgery, angioplasty, aortic valve replacement, and more, except for cases included in the "Life Stent Program."
What treatments are covered under neurosurgery?

It covers neurosurgical surgeries, both open heart surgery and percutaneous coronary intervention.
How are chronic diseases treated under the insurance coverage?

Acute conditions of chronic diseases are treated, and surgical treatment for chronic diseases is covered.
Is there an annual health check-up included?

Yes, a routine annual health check-up is provided, including specialist consultations, laboratory examinations, and instrumental examinations.
Are traditional medicine practices covered?

Yes, acupuncture and aromatherapy are covered when scientific medicine cannot be applied.
Is personal accident insurance included?

Yes, there is a one-time compensation in case of the insured person's death due to an accident.
Are pre-existing illnesses covered under the insurance?

No, pre-existing illnesses are generally not covered unless specified otherwise in the contract or rules.
Can the insured choose any medical facility for the annual health check-up?

The check-ups can be conducted at specific medical facilities listed in the brochure.
What is the compensation limit for dental orthopedics?

The limit for dental orthopedics includes the cost of therapeutic or surgical treatment preceding orthopedic prosthesis.
What is the compensation amount for personal accident insurance?

The one-time compensation in case of death due to an accident is AMD 1,000,000.



Q1: What medical services are covered under the health insurance plan?
A1: The plan covers a wide range of services, including ambulance services, outpatient treatments, in-patient treatments, vertebrology, physiotherapy, ophthalmology, dentistry, pregnancy and childbirth, oncology, cardiology, neurosurgery, chronic diseases, annual health check-ups, traditional medicine, and personal accident insurance.

Q2: Are emergency ambulance services included in the health plan?
A2: Yes, the plan provides full range services of ambulance, including urgent medical transfers within the Republic of Armenia and Nagorno-Karabakh, and 24/7 medical consultations with professionals.

Q3: What outpatient treatments are covered by the health plan?
A3: Outpatient treatments covered by the plan include medical consultations, laboratory and instrumental diagnostics, medical interventions, surgical interventions, treatment for traumas, burns, infectious diseases, poisonings, and more. It also covers the cost of prescribed medications and medical supplies.

Q4: Are dental services covered by the health plan?
A4: Yes, the plan includes coverage for various dental services such as X-rays, tooth crown reconstruction, endodontics, periodontitis treatment, dental surgeries, orthopedics, implantology, and more.

Q5: Does the health plan cover pregnancy and childbirth expenses?
A5: Yes, the plan covers both normal and pathological pregnancy expenses that are outside the scope of state order. It also provides coverage for child delivery, including cesarean sections.

Q6: Is cancer treatment covered under the health plan?
A6: Yes, the plan covers surgical treatment of cancerous diseases and non-surgical complex in-patient treatments such as chemotherapy and radiation therapy.

Q7: Are annual health check-ups included in the health plan?
A7: Yes, the plan provides for routine annual health check-ups, which include specialist consultations, laboratory examinations, and instrumental examinations.

Q8: Does the health plan cover traditional medicine?
A8: Yes, in cases where scientific medicine may not be applied, the plan covers reasonable and customary charges for services like acupuncture and aromatherapy.

Q9: Is personal accident insurance included in the health plan?
A9: Yes, the plan provides a one-time compensation in the event of the insured person's death as a result of an accident.

Q10: Are cosmetic procedures covered by the health plan?
A10: No, cosmetic procedures are generally not covered under the health plan unless they are medically necessary.

Q11: Can I get reimbursement for prescribed medications?
A11: Yes, the plan provides reimbursement for the cost of prescribed medications both for outpatient and in-patient treatments.

Q12: Are vaccinations for traveling abroad covered by the health plan?
A12: Yes, the plan covers vaccinations for traveling abroad.

Q13: Is infertility treatment covered by the health plan?
A13: The coverage for infertility treatment would depend on the specific terms and conditions of the health plan. It is recommended to review the plan's documentation or contact the insurer for more information.

Q14: Are mental health services covered by the health plan?
A14: The plan includes a one-time consultation with a psychologist for each insured person. However, the extent of mental health coverage may vary, and it is advisable to review the plan's details or contact the insurer for specific information on mental health services.

Q15: Are there any waiting periods for specific services under the health plan?
A15: The plan may have waiting periods for certain services. For example, there is a 9-month waiting period for coverage of child delivery for new insured people. It is recommended to refer to the plan's terms and conditions or consult with the insurer for detailed information on waiting periods.

Q16: Can I choose any medical facility or provider for treatment under the health plan?
A16: The health plan may have a network of preferred medical facilities or providers. It is advisable to check the plan's provider directory or consult with the insurer to ensure coverage at the desired facility or with a specific provider.

Q17: Are pre-existing conditions covered by the health plan?
A17: Pre-existing conditions are generally not covered under the health plan, unless specified otherwise in the contract or rules. It is essential to review the plan's terms and conditions or discuss with the insurer to understand the coverage for pre-existing conditions.

Q18: What is the procedure for claiming reimbursement under the health plan?
A18: To claim reimbursement, you would typically need to submit the necessary documentation, such as invoices, medical reports, and prescriptions, to the insurer. It is advisable to follow the specific claim submission process outlined by the insurer.

Q19: Are preventive screenings covered by the health plan?
A19: Yes, the health plan often includes coverage for preventive screenings, such as mammograms, Pap smears, and colonoscopies. These screenings aim to detect potential health issues early on and prevent the development of serious conditions.

Q20: Can I add dependents to my health plan?
A20: Yes, most health plans allow you to add dependents, such as spouse, children, or other eligible family members, to your coverage. The specific rules and procedures for adding dependents may vary, so it is recommended to consult with the insurer for the necessary steps to include them in your plan.

Here is a summary of what is covered under the health insurance:

Ambulance services, including urgent medical transfers within Armenia and Nagorno-Karabakh, and 24/7 medical consultations with professionals.
Out-patient treatment, which includes medical consultations, laboratory and instrumental diagnostics, medical interventions, surgical interventions, treatment for traumas, burns, infectious diseases, poisonings, vaccinations for travel, psychological consultations, home visit services, and more.
In-patient treatment, covering urgent therapeutic or surgical diseases, surgical treatments, diagnostics, treatment for infectious diseases, traumas, burns, acute poisonings, intensive therapy, and reanimation.
Vertebral treatment, including drug treatment, physiotherapy methods, and surgical treatment of spinal hernia.
Physiotherapy, covering electrotherapy, post-traumatic physiotherapeutic and rehabilitation treatment, excluding spine-related cases covered under vertebral treatment.
Ophthalmology, providing surgical and non-surgical treatment for eye diseases, as well as reimbursement for optical lenses and glasses.
Dentistry, with coverage for various dental services such as X-rays, crown reconstruction, endodontics, periodontitis treatment, surgery, dental orthopedics, and implantology.
Pregnancy and childbirth coverage, including normal and pathological pregnancy expenses, as well as coverage for child delivery (including cesarean section) with a waiting period for new insured individuals.
Oncology, covering surgical and non-surgical in-patient treatment of cancerous diseases.
Cardiology, including cardio-surgery, coronary interventions, angioplasty, aortic valve replacement, with some exclusions if covered under the "Life Stent Program" of the Ministry of Health.
Neurosurgery, providing coverage for neurosurgical surgeries.
Chronic diseases treatment, covering acute conditions, non-surgical services, surgical treatment for full recovery, and cost compensation for dynamic control of chronic diseases.
Annual health check-up, offering routine check-ups with specialist consultations, laboratory and instrumental examinations, and annual health screenings at specific medical facilities.
Traditional medicine, covering acupuncture and aromatherapy when scientific medicine cannot be applied.
Personal accident insurance, providing a one-time compensation in case of the insured person's death due to an accident.
General exceptions, including pre-existing illnesses, injuries, and other conditions or services not covered by insurance.


the questions that are outside health insurance context answer - Sorry but that is outside my scope of knowledge

the questions that are outside health insurance context answer - Sorry but that is outside my scope of knowledge
the questions that are outside health insurance context answer - Sorry but that is outside my scope of knowledge
the questions that are outside health insurance context answer - Sorry but that is outside my scope of knowledge
the questions that are outside health insurance context answer - Sorry but that is outside my scope of knowledge
the questions that are outside health insurance context answer - Sorry but that is outside my scope of knowledge
the questions that are outside health insurance context answer - Sorry but that is outside my scope of knowledgethe
 questions that are outside health insurance context answer - Sorry but that is outside my scope of knowledge
"""

In [ ]:
with open('./document.txt', 'w', encoding='utf-8') as file:
    file.write(text1)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader('./').load_data()
documents

[Document(id_='93c481f9-1105-43a1-9bc8-33acb167d34b', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='3b1dd167b30db6a5611c350469479a3395b833c478bf45f2d010e0cc2f437b0d', text='\n          Dear Customer,\n\n\n\n\nWe are happy to inform you that “INGO ARMENIA” Insurance CJSC continues to stay\nthe provider of health insurance of Simonyan Educational Foundation and its affiliated\ncompanies employees.\n\n\nThe health insurance coverage, application procedure, the list of partner medical\nfacilities are presented in this brochure.\n\n\nContact us and we will take care of your health․\n\n\nPlease get acquainted with health insurance’s terms and conditions.\n\n\n\nPlease call INGO ARMENIA 24/7 Call Center in case of questions or for getting further\ninformation:\n\n\n\n\n                                  +374 (10) 59-21-21\n\nWHAT IS COVERED\n\n  1. AMBULANCE\n  1.1. Full range services of ambulance\n  1.2. Urgent medical tr

In [ ]:
#documents = SimpleDirectoryReader('data').load_data()

# from llama_index import Document

# text_list = [text1]

# documents = [Document(t) for t in text_list]

In [ ]:
# set number of output tokens
num_output = 500
# set maximum input size
max_input_size = 512
# set maximum chunk overlap
max_chunk_overlap = 1


prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)


In [ ]:
#index = GPTSimpleVectorIndex(documents, embed_model=embed_model, llm_predictor=llm_predictor)

#index = GPTListIndex.from_documents(documents, embed_model=embed_model, llm_predictor=llm_predictor)

#index.save_to_disk('index.json')

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model)
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)


In [ ]:
import logging

logging.getLogger().setLevel(logging.CRITICAL)

In [ ]:
query_engine = index.as_query_engine()
prompt = " what dental services does my insurance cover? "
response = query_engine.query(prompt + 'is the question related to your custom knowledge?')
print(response)


Token indices sequence length is longer than the specified maximum sequence length for this model (1395 > 512). Running this sequence through the model will result in indexing errors


yes


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# this cell should be run only once, to install the telebot library
! pip install pytelegrambotapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-4.12.0-py3-none-any.whl size=213956 sha256=1e1c3d24c0a9588458537eb0c045d48173e35c55ac592e858641f20c3fe4c2a2
  Stored in directory: /root/.cache/pip/wheels/7e/ba/82/f3ab5bc48525778633bccc741c0424677ed3435736221819f4
Successfully built pytelegrambotapi


In [ ]:
# here we create the simplest possible bot

import telebot


API_TOKEN = '6156036993:AAFP6oS_-bBJdLv1YDWUTto0PdCoZYaEihU'
bot = telebot.TeleBot(API_TOKEN)


@bot.message_handler(func=lambda message: True)
def generate_response(message):
    input_text = message.text




    # # # Tokenize the input text
    # input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # # Generate the response using the T5 model
    # with torch.no_grad():
    #     output_ids = model.generate(input_ids)

    # Decode the generated response
    query_engine = index.as_query_engine()
    test = "is the question related to your custom knowledge?"
    response_test = query_engine.query(input_text + test)
    if str(response_test) == 'yes':
      response = query_engine.query(input_text)
    else:
      response = 'Sorry, I’m afraid I cannot fulfill that request as the provided CONTEXT INFORMATION does not relate to health insurance'
    # response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    bot.reply_to(message, response)

In [ ]:
# and here we actually run it
bot.polling()

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# pip install deep_translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import os
# import json
# from deep_translator import GoogleTranslator

In [ ]:
# translator = GoogleTranslator(source='auto', target='en')

In [ ]:
# print(translator.translate("Կարո՞ղ եմ մեջքիս ռենտգեն անել:"))

Can I have an X-ray of my back?


In [ ]:
# query_engine = index.as_query_engine()

# translator_to_en = GoogleTranslator(source='auto', target='en')
# prompt_hy = "Կարո՞ղ եմ մեջքիս ռենտգեն անել:"
# print(prompt_hy)
# prompt_en = translator_to_en.translate(prompt_hy)
# print(prompt_en)

# response_en = query_engine.query(prompt_en)
# print(response_en)
# translator_to_hy = GoogleTranslator(source='auto', target='hy')
# response_hy = translator_to_hy.translate(str(response_en))
# print(response_hy)

Կարո՞ղ եմ մեջքիս ռենտգեն անել:
Can I have an X-ray of my back?
yes
այո
